<a href="https://colab.research.google.com/github/monash-tp37-project/iteration_1_model/blob/master/bert_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and use pretrained BERT model for news classification

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#Installing the Hugging Face PyTorch Interface for Bert
# !pip install pytorch-pretrained-bert pytorch-nlp
!pip install -q transformers
!pip3 install keybert
!pip3 install keyphrase-vectorizers

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
thinc 8.0.15 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
spacy 3.3.0 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typin

In [9]:
import os
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

# libraries for keywords
from keybert import KeyBERT
import keyphrase_vectorizers
from keyphrase_vectorizers import KeyphraseCountVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
current_dir = os.getcwd()
current_dir

'/content'

In [11]:
#import torch
#from transformers import AutoTokenizer, AutoModelForSequenceClassification

# read in trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/tokenizer_150k_512_10k_fake")
model = BertForSequenceClassification.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/model_150k_512_16epoch_10k_fake")

OSError: ignored

In [ ]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in english_stop_words:
            result.append(token)
            
    return result

In [ ]:
# save list of all stopwords
english_stop_words = stopwords.words('english')

In [ ]:
sequences = [preprocess(("Previously unreported surveillance video captured one such effort in August in the rural Colorado town of Kiowa. Footage obtained by Reuters through a public-records request shows Elbert County Clerk Dallas Schroeder, the county’s top election official, fiddling with cables and typing on his phone as he copied computer drives containing sensitive voting information.")]

# tokenize text input
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

In [ ]:
with torch.no_grad():
  # make predictions
  output = model(**tokens)

  # using a softmax activation function get discrete predictions
  predictions = F.softmax(output.logits, dim=1)
  labels = int(torch.argmax(predictions, dim=1))

  # using a sigmoid function get continous predictions 
  label = torch.sigmoid(predictions).tolist()[0]
  reliable = round(label[0],2)
  unreliable = round(label[1],2)
  print(f'Your news has a {unreliable}% probability of being unreliable and a {reliable}% of being reliable.')

  if reliable < unreliable:

    print('Therefore, our model predicts your news is likely to be unreliable.')

  else:

    print('Therefore, our model predicts your news is likely to be reliable.')

Your news has a 0.5% probability of being unreliable and a 0.73% of being reliable.
Therefore, our model predicts your news is likely to be reliable.


In [ ]:
kw_model = KeyBERT()

In [ ]:
keywords = kw_model.extract_keywords(docs=sequences, vectorizer=KeyphraseCountVectorizer())

/usr/local/lib/python3.7/dist-packages/keybert/_model.py:131: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  "Although extracting keywords for multiple documents is faster "
1it [00:00, 424.48it/s]


In [ ]:
[x[0] for x in sorted(keywords[0], key=lambda x: x[1], reverse=True)][0:3]

['elbert county clerk dallas schroeder',
 'sensitive voting information',
 'county']